In [18]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

import spacy

# Charger le modèle français de spacy
nlp = spacy.load('fr_core_news_sm')




[nltk_data] Downloading package stopwords to /home/lahad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lahad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Séparation l'ensemble d'entraînement en 2 parties: entrainement et validation

In [19]:
# Charger les données
# data = pd.read_csv("../data/train.csv")

# train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# train_data.to_csv("../data/train_split.csv", index=False)
# val_data.to_csv("../data/validation_split.csv", index=False)


## Chargement des données

In [20]:
# Charger les données
train_data = pd.read_csv("../data/train_split.csv")
val_data = pd.read_csv("../data/validation_split.csv")
test_data = pd.read_csv("../data/test.csv")

train_data.head()

,doc_id,titre,type,difficulte,cout,ingredients,recette
0,recette_170277.xml,Langue de boeuf sauce piquante aux cornichons,Plat principal,Facile,Moyen,"- 1 langue de 1,5 kg - 20 cl de vinaigre d'alc...",Faire tremper la langue toute une nuit dans un...
1,recette_21519.xml,Tarte aux raisins et amandes,Dessert,Facile,Moyen,- 1 pâte brisée - 150 g de raisins bien mûrs -...,Garnissez une tourtière avec la pâte étalée. F...
2,recette_72746.xml,Meringues bien sèches,Dessert,Très facile,Bon marché,- 2 blancs d'oeufs - 60 g de sucre en poudre -...,"Préchauffez le four à 150°, thermostat 5. Ajou..."
3,recette_38858.xml,"Sandwich jambon, menthe et feta",Plat principal,Très facile,Moyen,- 3 tranches de pain complet ou de seigle ou d...,Faire la petit sauce à la menthe : mêler crème...
4,recette_167836.xml,Salade de mâche et lardons,Entrée,Très facile,Bon marché,- 300 g de mâche - quelques feuilles de roquet...,Faire revenir les lardons et les déposer chaud...


## Statistiques sur les données

In [21]:
# # Nombre de documents dans chaque ensemble
# num_train_docs = len(train_data)
# num_val_docs = len(val_data)
# num_test_docs = len(test_data)

# print("Nombre de documents dans chaque ensemble :")
# print("Entraînement :", num_train_docs)
# print("Validation :", num_val_docs)
# print("Test :", num_test_docs)

# # Nombre de documents dans chaque ensemble pour chaque classe
# train_label_counts = train_data['type'].value_counts()
# val_label_counts = val_data['type'].value_counts()
# test_label_counts = test_data['type'].value_counts()

# print("\nNombre de documents par classe dans chaque ensemble :")
# print("Ensemble d'entraînement :")
# print(train_label_counts)
# print("\nEnsemble de validation :")
# print(val_label_counts)
# print("\nEnsemble de test :")
# print(test_label_counts)

# Pourcentage de documents dans chaque ensemble pour chaque classe
train_label_percentages = (train_data['type'].value_counts(normalize=True) * 100).round(2)
val_label_percentages = (val_data['type'].value_counts(normalize=True) * 100).round(2)
test_label_percentages = (test_data['type'].value_counts(normalize=True) * 100).round(2)

print("\nPourcentage de documents par classe dans chaque ensemble :")
print("Ensemble d'entraînement :")
print(train_label_percentages)
print("\nEnsemble de validation :")
print(val_label_percentages)
print("\nEnsemble de test :")
print(test_label_percentages)



Pourcentage de documents par classe dans chaque ensemble :
Ensemble d'entraînement :
type
Plat principal    46.54
Dessert           30.43
Entrée            23.03
Name: proportion, dtype: float64

Ensemble de validation :
type
Plat principal    46.41
Dessert           29.10
Entrée            24.49
Name: proportion, dtype: float64

Ensemble de test :
type
Plat principal    46.40
Dessert           29.32
Entrée            24.28
Name: proportion, dtype: float64


### Separer les données en attributs et étiquettes


In [22]:
# Prétraitement des données d'entraînement
train_data['text'] = train_data['titre'] + " " + train_data['recette']
train_data = train_data[['text', 'type']]

# Prétraitement des données de validation
val_data['text'] = val_data['titre'] + " " + val_data['recette']
val_data = val_data[['text', 'type']]

# Prétraitement des données de test
test_data['text'] = test_data['titre'] + " " + test_data['recette']
test_data = test_data[['text', 'type']]

# Diviser les données en attributs et étiquettes
X_train = train_data['text']
y_train = train_data['type']
X_val = val_data['text']
y_val = val_data['type']
X_test = test_data['text']
y_test = test_data['type']

###  Nettoyage des donnees 

In [23]:
import re
from nltk.tokenize import RegexpTokenizer
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer

stopwords_fr = set(stopwords.words('french'))
stemmer = FrenchStemmer()
tokenizer = RegexpTokenizer(r'\w+')

def clean_text(text):
    # Tokenization and lowercase conversion
    words = simple_preprocess(text, min_len=2)
    words = [word for word in words if word not in stopwords_fr]
    doc = nlp(" ".join(words))
    words = [stemmer.stem(word) for word in words]
    words = [token.lemma_ for token in doc]
    
    cleaned_text = ' '.join(words)

    return cleaned_text

# Appliquer le nettoyage sur les données 
X_train = X_train.apply(clean_text)
X_val = X_val.apply(clean_text)
X_test = X_test.apply(clean_text)



## Baseline

In [24]:

# Créer un classificateur baseline
dummy = DummyClassifier(strategy='stratified', random_state=42)
dummy.fit(X_train, y_train)

# # Prédire sur les données de validation
# y_pred_val = dummy.predict(X_val)

# # Afficher le rapport de classification pour les données de developpement
# print("Classification Report for Validation Data:")
# print(classification_report(y_val, y_pred_val))

# Prédire sur les données de test
y_pred_test = dummy.predict(X_test)

# Afficher le rapport de classification pour les données de test
print("Classification Report for Test Data:")
print(classification_report(y_test, y_pred_test,digits=4))


Classification Report for Test Data:
                precision    recall  f1-score   support

       Dessert     0.3088    0.3194    0.3140       407
        Entrée     0.2656    0.2522    0.2588       337
Plat principal     0.4930    0.4953    0.4942       644

      accuracy                         0.3847      1388
     macro avg     0.3558    0.3557    0.3557      1388
  weighted avg     0.3838    0.3847    0.3842      1388



#### Run2: TF-IDF + SVM

In [25]:
# Vectorisation des textes
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

# Création du modèle SVM avec noyau gaussien (RBF)
svm_rbf = SVC(kernel='rbf', C=5, random_state=42)


# Entraînement du modèle
svm_rbf.fit(X_train_tfidf, y_train)

# Prédiction sur les données de developpement
# y_pred_val = svm_rbf.predict(X_val_tfidf)

# # Evaluation sur les données de developpement
# accuracy_val = accuracy_score(y_val, y_pred_val)
# precision_val = precision_score(y_val, y_pred_val, average='weighted')
# recall_val = recall_score(y_val, y_pred_val, average='weighted')
# f1_val = f1_score(y_val, y_pred_val, average='weighted')
# conf_matrix_val = confusion_matrix(y_val, y_pred_val)
# class_report_val = classification_report(y_val, y_pred_val,digits=3)

# Affichage des résultats sur les données de developpement
# print("Evaluation Metrics sur les donnees de developpement:")
# print(f"Accuracy: {accuracy_val}")
# print(f"Precision: {precision_val}")
# print(f"Recall: {recall_val}")
# print(f"F1 Score: {f1_val}")
# print("Confusion Matrix:")
# print(conf_matrix_val)
# print("Classification Report:")
# print(class_report_val)

# Prédiction sur les données de test
y_pred_test = svm_rbf.predict(X_test_tfidf)

# Evaluation sur les données de test
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
class_report_test = classification_report(y_test, y_pred_test,digits=3)

# Affichage des résultats sur les données de test
print("\nEvaluation Metrics sur les donnees de Test:")
print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")
print("Confusion Matrix:")
print(conf_matrix_test)
print("Classification Report:")
print(class_report_test)



Evaluation Metrics sur les donnees de Test:
Accuracy: 0.8731988472622478
Precision: 0.8716080572000864
Recall: 0.8731988472622478
F1 Score: 0.8722572692756911
Confusion Matrix:
[[405   1   1]
 [  2 244  91]
 [  5  76 563]]
Classification Report:
                precision    recall  f1-score   support

       Dessert      0.983     0.995     0.989       407
        Entrée      0.760     0.724     0.742       337
Plat principal      0.860     0.874     0.867       644

      accuracy                          0.873      1388
     macro avg      0.868     0.864     0.866      1388
  weighted avg      0.872     0.873     0.872      1388



### Run3: Word2Vec + SVM

In [26]:
# Modèle Word2Vec
corpus = [sentence.split() for sentence in X_train]
word2vec_model = Word2Vec(corpus, vector_size=150, window=10, min_count=1, workers=1, sg=1)

# Calcul des vecteurs de mots moyens pour chaque document
def document_vector(word2vec_model, doc):
    """Calculer le vecteur moyen pour un document en utilisant les vecteurs de mots Word2Vec"""
    # Filtrer les mots absents dans le vocabulaire
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) != 0:
        return np.mean(word2vec_model.wv[doc], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Préparation des données de test pour Word2Vec
X_train_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_train])
X_val_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_val])
X_test_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_test])

# Modèle SVM
svm_classifier = SVC(kernel='rbf', C=5, random_state=42)
svm_classifier.fit(X_train_word2vec, y_train)

# Prédiction sur les données de test
y_pred_test = svm_classifier.predict(X_test_word2vec)

accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
class_report_test = classification_report(y_test, y_pred_test,digits=3)

print("\nEvaluation Metrics sur les donnees de Test:")
print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")
print("Confusion Matrix:")
print(conf_matrix_test)
print("Classification Report:")
print(class_report_test)


# # Evaluation
# Prédiction sur les données de developpement
# y_pred_val = svm_classifier.predict(X_val_word2vec)
# accuracy_val = accuracy_score(y_val, y_pred_val)
# precision_val = precision_score(y_val, y_pred_val, average='weighted')
# recall_val = recall_score(y_val, y_pred_val, average='weighted')
# f1_val = f1_score(y_val, y_pred_val, average='weighted')
# conf_matrix_val = confusion_matrix(y_val, y_pred_val)
# class_report_val = classification_report(y_val, y_pred_val,digits=4)
# Affichage des résultats
# print("Evaluation Metrics sur les donnees de  developpement :")
# print(f"Accuracy: {accuracy_val}")
# print(f"Precision: {precision_val}")
# print(f"Recall: {recall_val}")
# print(f"F1 Score: {f1_val}")
# print("Confusion Matrix:")
# print(conf_matrix_val)
# print("Classification Report:")
# print(class_report_val)


Evaluation Metrics sur les donnees de Test:
Accuracy: 0.8681556195965417
Precision: 0.8658581742307588
Recall: 0.8681556195965417
F1 Score: 0.8648599185905296
Confusion Matrix:
[[403   2   2]
 [  3 220 114]
 [  4  58 582]]
Classification Report:
                precision    recall  f1-score   support

       Dessert      0.983     0.990     0.987       407
        Entrée      0.786     0.653     0.713       337
Plat principal      0.834     0.904     0.867       644

      accuracy                          0.868      1388
     macro avg      0.867     0.849     0.856      1388
  weighted avg      0.866     0.868     0.865      1388



### Run4: CountVectorizer + SVM

In [27]:
# Vectorisation des textes
vectorizer = CountVectorizer()

# Transformation des données textuelles en vecteurs de compte
X_train_counts = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

# Création et entraînement du modèle SVM
svm_classifier = SVC(kernel="rbf", C=10, random_state=42)
svm_classifier.fit(X_train_counts, y_train)

# Prédiction sur les données de test
y_pred_test = svm_classifier.predict(X_test_counts)

# Evaluation sur les données de test
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
class_report_test = classification_report(y_test, y_pred_test,digits=3)

# Affichage des résultats sur les données de test
print("\nEvaluation Metrics sur les donnees de Tests:")
print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")
print("Confusion Matrix:")
print(conf_matrix_test)
print("Classification Report:")
print(class_report_test)




# # Prédiction sur les données de developpement
# y_pred_val = svm_classifier.predict(X_val_counts)

# # Evaluation sur les données de developpement
# accuracy_val = accuracy_score(y_val, y_pred_val)
# precision_val = precision_score(y_val, y_pred_val, average='weighted')
# recall_val = recall_score(y_val, y_pred_val, average='weighted')
# f1_val = f1_score(y_val, y_pred_val, average='macro')
# conf_matrix_val = confusion_matrix(y_val, y_pred_val)
# class_report_val = classification_report(y_val, y_pred_val,digits=4)

# # Affichage des résultats sur les données de developpement
# print("Evaluation Metrics sur les donnees de developpement :")
# print(f"Accuracy: {accuracy_val}")
# print(f"Precision: {precision_val}")
# print(f"Recall: {recall_val}")
# print(f"F1 Score: {f1_val}")
# print("Confusion Matrix:")
# print(conf_matrix_val)
# print("Classification Report:")
# print(class_report_val)


Evaluation Metrics sur les donnees de Tests:
Accuracy: 0.8652737752161384
Precision: 0.8628157915033674
Recall: 0.8652737752161384
F1 Score: 0.863688358799296
Confusion Matrix:
[[404   1   2]
 [  5 235  97]
 [  6  76 562]]
Classification Report:
                precision    recall  f1-score   support

       Dessert      0.973     0.993     0.983       407
        Entrée      0.753     0.697     0.724       337
Plat principal      0.850     0.873     0.861       644

      accuracy                          0.865      1388
     macro avg      0.859     0.854     0.856      1388
  weighted avg      0.863     0.865     0.864      1388



In [28]:
# from sklearn.model_selection import GridSearchCV

# parameters = {'kernel':('linear', 'rbf'), 'C':[1,10]}
# svc = SVC(random_state=42)
# clf = GridSearchCV(svc, parameters)
# clf.fit(X_train_tfidf, y_train)
# #Print the best parameters
# print("Best parameters: ", clf.best_params_)
# # Use the best estimator to make predictions
# y_pred = clf.best_estimator_.predict(X_test_tfidf)

# # Evaluation
# print(classification_report(y_testt, y_pred))